In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

In [9]:
df = pd.read_parquet(r"data/joind_output.parquet", engine="pyarrow")

In [10]:
df

,neid,LOCALCELLID,dateday,DLEARFCN,cellreselpriority_x,qrxlevmin_x,snonintrasearch,thrshservlow,cellreselpriority_y,qoffsetfreq,...,TIM_PRB_UTIL_MEAN_UL_DEN (Unid),TIM_PRB_UTIL_MEAN_UL_NUM (Unid),TIM_THROU_USER_PDCP_DL (Kbps),TIM_THROU_USER_PDCP_DL_DEN (s),TIM_THROU_USER_PDCP_DL_NUM (Kb),TIM_THROU_USER_PDCP_UL (Kbps),TIM_THROU_USER_PDCP_UL_DEN (s),TIM_THROU_USER_PDCP_UL_NUM (Kb),TIM_USERS_RRC_CONN_MEAN_SUM (Unid),TIM_VOLUME_DADOS_DL_ALLOP (KB)
0,18NLITAG01,1,2023-08-21,100,7,-62,15,0,7,dB-22,...,50.0,949.0,9455.377060,564960.0,5.341910e+09,839.193198,426603.0,358002336.0,6.847,680549.164062
1,18NLITAG01,1,2023-08-21,100,7,-62,15,0,7,dB-22,...,50.0,871.1,12700.061449,178750.0,2.270136e+09,627.378004,282349.0,177139552.0,6.024,303761.855469
2,18NLITAG01,1,2023-08-21,100,7,-62,15,0,7,dB-22,...,50.0,788.8,10109.848984,65993.0,6.671793e+08,173.679601,336718.0,58481048.0,4.944,86050.669922
3,18NLITAG01,1,2023-08-21,100,7,-62,15,0,7,dB-22,...,50.0,690.4,16529.563141,14238.0,2.353479e+08,314.652147,255292.0,80328176.0,3.606,32611.041016
4,18NLITAG01,1,2023-08-21,100,7,-62,15,0,7,dB-22,...,50.0,799.5,9299.013731,109313.0,1.016503e+09,364.658493,547468.0,199638856.0,5.150,130931.255859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3905365,SLRJZ602,9,2023-10-01,9410,7,-60,15,1,5,dB12,...,75.0,466.2,63681.882353,17.0,1.082592e+06,1684.788396,2344.0,3949144.0,1.003,942.801758
3905366,SLRJZ602,9,2023-10-01,9410,7,-60,15,1,5,dB12,...,75.0,466.2,3600.000000,1.0,3.600000e+03,1677.964082,2283.0,3830792.0,1.017,724.338867
3905367,SLRJZ602,9,2023-10-01,9410,7,-60,15,1,5,dB12,...,75.0,465.5,26472.000000,6.0,1.588320e+05,2215.947977,1384.0,3066872.0,1.000,602.082031
3905368,SLRJZ602,9,2023-10-01,9410,7,-60,15,1,5,dB12,...,75.0,466.1,3960.000000,2.0,7.920000e+03,1607.312633,2351.0,3778792.0,1.006,782.392578


In [11]:
na_counts = df.isna().sum()
na_counts_df = na_counts.reset_index()
na_counts_df.columns = ['Coluna', 'Qtd_NaNs']
print(na_counts_df)

                                Coluna  Qtd_NaNs
0                                 neid         0
1                          LOCALCELLID         0
2                              dateday         0
3                             DLEARFCN         0
4                  cellreselpriority_x         0
..                                 ...       ...
56       TIM_THROU_USER_PDCP_UL (Kbps)    311579
57      TIM_THROU_USER_PDCP_UL_DEN (s)    234288
58     TIM_THROU_USER_PDCP_UL_NUM (Kb)    234288
59  TIM_USERS_RRC_CONN_MEAN_SUM (Unid)    234288
60      TIM_VOLUME_DADOS_DL_ALLOP (KB)    234288

[61 rows x 2 columns]


In [12]:
df = df.drop(columns=["TIM_BUFFER_MEAN_USER_DL_VOLTE (Unid)", "TIM_IRAT_HO_SUCC_OUT (%)"])
df = df.dropna(axis=0)


In [13]:
df

,neid,LOCALCELLID,dateday,DLEARFCN,cellreselpriority_x,qrxlevmin_x,snonintrasearch,thrshservlow,cellreselpriority_y,qoffsetfreq,...,TIM_PRB_UTIL_MEAN_UL_DEN (Unid),TIM_PRB_UTIL_MEAN_UL_NUM (Unid),TIM_THROU_USER_PDCP_DL (Kbps),TIM_THROU_USER_PDCP_DL_DEN (s),TIM_THROU_USER_PDCP_DL_NUM (Kb),TIM_THROU_USER_PDCP_UL (Kbps),TIM_THROU_USER_PDCP_UL_DEN (s),TIM_THROU_USER_PDCP_UL_NUM (Kb),TIM_USERS_RRC_CONN_MEAN_SUM (Unid),TIM_VOLUME_DADOS_DL_ALLOP (KB)
48168,18NLITAG01,1,2023-09-18,100,7,-60,15,1,7,dB14,...,50.0,1198.5,20400.410918,516152.0,1.052971e+10,1149.416712,514758.0,591671448.0,10.473,1.382850e+06
48169,18NLITAG01,1,2023-09-18,100,7,-60,15,1,7,dB14,...,50.0,922.9,24518.369100,426405.0,1.045476e+10,1022.926265,294337.0,301085048.0,7.162,1.401935e+06
48170,18NLITAG01,1,2023-09-18,100,7,-60,15,1,7,dB14,...,50.0,837.3,13574.512948,31278.0,4.245836e+08,343.760411,263685.0,90644464.0,4.879,6.673431e+04
48171,18NLITAG01,1,2023-09-18,100,7,-60,15,1,7,dB14,...,50.0,749.1,8917.557690,163989.0,1.462381e+09,592.474995,221897.0,131468424.0,4.759,2.014159e+05
48172,18NLITAG01,1,2023-09-18,100,7,-60,15,1,7,dB14,...,50.0,840.5,11995.291205,72873.0,8.741329e+08,2757.268815,232201.0,640240576.0,4.711,1.292979e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3905239,SLRJZ602,9,2023-09-28,9410,7,-60,15,1,5,dB12,...,75.0,637.8,29248.548841,311041.0,9.097498e+09,6308.453913,121759.0,768111040.0,11.069,1.364235e+06
3905240,SLRJZ602,9,2023-09-28,9410,7,-60,15,1,5,dB12,...,75.0,628.9,36879.506260,117973.0,4.350786e+09,8306.879726,93545.0,777067064.0,8.534,7.215825e+05
3905241,SLRJZ602,9,2023-09-28,9410,7,-60,15,1,5,dB12,...,75.0,549.8,27522.144769,99013.0,2.725050e+09,3226.385274,53619.0,172995552.0,7.708,4.768550e+05
3905243,SLRJZ602,9,2023-09-28,9410,7,-60,15,1,5,dB12,...,75.0,516.9,33667.389522,32106.0,1.080925e+09,5939.392006,33849.0,201042480.0,5.422,2.131735e+05


In [15]:
df["qoffsetfreq"] = df["qoffsetfreq"].str.replace("dB", "", regex=False).astype(float)